### Load and Inspect the Dataset

In [4]:
import pandas as pd

In [ ]:
train_df = pd.read_csv('arxiv_train.csv')
test_df = pd.read_csv('arxiv_test.csv')

### starting preprocessing you lame ass nigga

- checking null values

In [6]:
print("Train missing values:\n", train_df.isnull().sum())
print("Test missing values:\n", test_df.isnull().sum())

Train missing values:
 Unnamed: 0    0
abstract      0
label         0
dtype: int64
Test missing values:
 Unnamed: 0    0
abstract      0
label         0
dtype: int64


- removing na

In [7]:
train_df.dropna(subset=['abstract', 'label'], inplace=True)
test_df.dropna(subset=['abstract', 'label'], inplace=True)

- removing extra spaces, new lines, tabs

In [ ]:
import re

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    text = text.lower()
    return text

train_df['clean_abstract'] = train_df['abstract'].apply(clean_text)
test_df['clean_abstract'] = test_df['abstract'].apply(clean_text)


In [9]:
train_df.head()

,Unnamed: 0,abstract,label,clean_abstract
0,31716,Automatic meeting analysis is an essential f...,eess,automatic meeting analysis is an essential fun...
1,89533,We propose a protocol to encode classical bi...,quant-ph,we propose a protocol to encode classical bits...
2,82700,A number of physically intuitive results for...,quant-ph,a number of physically intuitive results for t...
3,78830,In the last decade rare-earth hexaborides ha...,physics,in the last decade rare-earth hexaborides have...
4,94948,We introduce the weak barycenter of a family...,stat,we introduce the weak barycenter of a family o...


- encoding labels as integers because its a multiclass problem

In [10]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

train_df['label_encoded'] = label_encoder.fit_transform(train_df['label'])
test_df['label_encoded'] = label_encoder.transform(test_df['label'])

# Save the mapping for later use
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label mapping:", label_mapping)


Label mapping: {'astro-ph': 0, 'cond-mat': 1, 'cs': 2, 'eess': 3, 'hep-ph': 4, 'hep-th': 5, 'math': 6, 'physics': 7, 'quant-ph': 8, 'stat': 9}


- Stratified Train/Dev Split

In [11]:
from sklearn.model_selection import train_test_split

# Stratified split on the encoded labels
train_texts, dev_texts, train_labels, dev_labels = train_test_split(
    train_df['clean_abstract'],
    train_df['label_encoded'],
    test_size=0.2,
    random_state=42,
    stratify=train_df['label_encoded']
)

# Optional sanity check: label distribution
print("Train label distribution:\n", train_labels.value_counts(normalize=True))
print("Dev label distribution:\n", dev_labels.value_counts(normalize=True))

Train label distribution:
 label_encoded
9    0.100687
3    0.100656
7    0.100281
4    0.100125
2    0.100062
0    0.099844
8    0.099828
5    0.099766
6    0.099469
1    0.099281
Name: proportion, dtype: float64
Dev label distribution:
 label_encoded
9    0.100687
3    0.100625
7    0.100312
4    0.100125
2    0.100062
8    0.099875
0    0.099812
5    0.099750
6    0.099500
1    0.099250
Name: proportion, dtype: float64


In [35]:
y_train = train_labels.values
y_dev = dev_labels.values
label_map = {'astro-ph': 0, 'cond-mat': 1, 'cs': 2, 'eess': 3, 'hep-ph': 4,
             'hep-th': 5, 'math': 6, 'physics': 7, 'quant-ph': 8, 'stat': 9}

# step 1 : text vectorization

- basic count vectorizer

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
count_vectorizer = CountVectorizer(max_features=10000, stop_words='english')

# Fit only on training data
X_train_count = count_vectorizer.fit_transform(train_texts)
X_dev_count = count_vectorizer.transform(dev_texts)

print("CountVectorizer shape (train):", X_train_count.shape)
print("CountVectorizer shape (dev):", X_dev_count.shape)


CountVectorizer shape (train): (64000, 10000)
CountVectorizer shape (dev): (16000, 10000)


- TF-IDF Vectorization

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=10000, stop_words='english')

# Fit only on training data
X_train_tfidf = tfidf_vectorizer.fit_transform(train_texts)
X_dev_tfidf = tfidf_vectorizer.transform(dev_texts)

print("TF-IDF shape (train):", X_train_tfidf.shape)
print("TF-IDF shape (dev):", X_dev_tfidf.shape)


TF-IDF shape (train): (64000, 10000)
TF-IDF shape (dev): (16000, 10000)


# building MLP

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from sklearn.metrics import classification_report

- converting data to tensors

# run this for BOW tensors

In [32]:
# Convert sparse matrices to dense and then to tensors
X_train_tensor = torch.tensor(X_train_count.toarray(), dtype=torch.float32)
X_dev_tensor = torch.tensor(X_dev_count.toarray(), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_dev_tensor = torch.tensor(y_dev, dtype=torch.long)

# Create datasets and loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
dev_dataset = TensorDataset(X_dev_tensor, y_dev_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=64)

# run this for TF-IDF tensors

In [37]:
X_train_tensor = torch.tensor(X_train_tfidf.toarray(), dtype=torch.float32)
X_dev_tensor = torch.tensor(X_dev_tfidf.toarray(), dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
dev_dataset = TensorDataset(X_dev_tensor, y_dev_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=64)


- definfing the model

In [38]:
class FeedforwardNN(nn.Module):
    def __init__(self, input_dim, hidden_dims, num_classes):
        super(FeedforwardNN, self).__init__()
        layers = []
        current_dim = input_dim
        for h in hidden_dims:
            layers.append(nn.Linear(current_dim, h))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.3))  # Regularization
            current_dim = h
        layers.append(nn.Linear(current_dim, num_classes))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [39]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = FeedforwardNN(input_dim=10000, hidden_dims=[128, 64], num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss:.4f}")


Epoch 1/5 - Loss: 831.5841
Epoch 2/5 - Loss: 439.9397
Epoch 3/5 - Loss: 336.5091
Epoch 4/5 - Loss: 258.9744
Epoch 5/5 - Loss: 196.4470


# results with BOW tensors 
- Epoch 1/5 - Loss: 706.6508
- Epoch 2/5 - Loss: 390.9463
- Epoch 3/5 - Loss: 277.9484
- Epoch 4/5 - Loss: 196.3589
- Epoch 5/5 - Loss: 142.3870

# rusults with TF-IDF tensors 
- Epoch 1/5 - Loss: 831.5841
- Epoch 2/5 - Loss: 439.9397
- Epoch 3/5 - Loss: 336.5091
- Epoch 4/5 - Loss: 258.9744
- Epoch 5/5 - Loss: 196.4470

In [41]:
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for batch_x, batch_y in dev_loader:
        batch_x = batch_x.to(device)
        outputs = model(batch_x)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(batch_y.numpy())

print(classification_report(all_labels, all_preds, target_names=label_map.keys()))

              precision    recall  f1-score   support

    astro-ph       0.95      0.93      0.94      1597
    cond-mat       0.81      0.78      0.79      1588
          cs       0.71      0.68      0.70      1601
        eess       0.76      0.82      0.79      1610
      hep-ph       0.93      0.93      0.93      1602
      hep-th       0.91      0.90      0.91      1596
        math       0.83      0.85      0.84      1592
     physics       0.70      0.67      0.68      1605
    quant-ph       0.84      0.87      0.86      1598
        stat       0.80      0.81      0.81      1611

    accuracy                           0.82     16000
   macro avg       0.82      0.83      0.82     16000
weighted avg       0.82      0.82      0.82     16000



# compraing BOW and TF

In [42]:
import pandas as pd

# Define the metrics for both models
data = {
    "Metric": [
        "Accuracy", "Macro F1", "Weighted F1", 
        "Astro-ph F1", "Cond-mat F1", "CS F1", "EESS F1", "Hep-ph F1", 
        "Hep-th F1", "Math F1", "Physics F1", "Quant-ph F1", "Stat F1"
    ],
    "CountVectorizer": [
        0.83, 0.83, 0.83,
        0.93, 0.80, 0.70, 0.79, 0.93,
        0.91, 0.85, 0.69, 0.86, 0.80
    ],
    "TF-IDF": [
        0.82, 0.82, 0.82,
        0.94, 0.79, 0.70, 0.79, 0.93,
        0.91, 0.84, 0.68, 0.86, 0.81
    ]
}

# Create the DataFrame
df = pd.DataFrame(data)

# Print the table
print(df.to_string(index=False))


     Metric  CountVectorizer  TF-IDF
   Accuracy             0.83    0.82
   Macro F1             0.83    0.82
Weighted F1             0.83    0.82
Astro-ph F1             0.93    0.94
Cond-mat F1             0.80    0.79
      CS F1             0.70    0.70
    EESS F1             0.79    0.79
  Hep-ph F1             0.93    0.93
  Hep-th F1             0.91    0.91
    Math F1             0.85    0.84
 Physics F1             0.69    0.68
Quant-ph F1             0.86    0.86
    Stat F1             0.80    0.81


# next steps (for said)

- You're in a great spot to move to Step 2: Pretrained Word Embeddings. Here's what you could try next:

- Replace BoW with averaged GloVe or word2vec embeddings for each abstract.

- Use mean/sum/max pooling over word embeddings to create document vectors.

- Feed those vectors into the same MLP structure for a fair comparison.